In [1]:
1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1721615106335_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [2]:
import io
import os
import re
import glob
import pandas as pd
import numpy as np

from datetime import datetime
import datetime as dt
import time
import boto3

import zipfile
import pytz

from datetime import date, timedelta
import gzip
import pandas as pd
from pandas.io.json import json_normalize 
import json
import gnupg
from io import StringIO
import boto3
import paramiko
import os
import requests
import logging
import os
import typing
from os.path import join, dirname
import time
import logging
from paramiko import SFTPClient, SFTPFile, Message, SFTPError, Transport
from paramiko.sftp import CMD_STATUS, CMD_READ, CMD_DATA

logger = logging.getLogger('demo')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib/python3.7/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/local/lib/python3.7/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
/usr/local/lib/python3.7/site-packages/paramiko/transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,

In [3]:
class _SFTPFileDownloader:
    """
    Helper class to download large file with paramiko sftp client with limited number of concurrent requests.
    """

    _DOWNLOAD_MAX_REQUESTS = 48
    _DOWNLOAD_MAX_CHUNK_SIZE = 0x8000

    def __init__(self, f_in: SFTPFile, f_out: typing.BinaryIO, callback=None):
        self.f_in = f_in
        self.f_out = f_out
        self.callback = callback

        self.requested_chunks = {}
        self.received_chunks = {}
        self.saved_exception = None

    def download(self):
        file_size = self.f_in.stat().st_size
        requested_size = 0
        received_size = 0

        while True:
            # send read requests
            while len(self.requested_chunks) + len(self.received_chunks) < self._DOWNLOAD_MAX_REQUESTS and \
                    requested_size < file_size:
                chunk_size = min(self._DOWNLOAD_MAX_CHUNK_SIZE, file_size - requested_size)
                request_id = self._sftp_async_read_request(
                    fileobj=self,
                    file_handle=self.f_in.handle,
                    offset=requested_size,
                    size=chunk_size
                )
                self.requested_chunks[request_id] = (requested_size, chunk_size)
                requested_size += chunk_size

            # receive blocks if they are available
            # note: the _async_response is invoked
            self.f_in.sftp._read_response()
            self._check_exception()

            # write received data to output stream
            while True:
                chunk = self.received_chunks.pop(received_size, None)
                if chunk is None:
                    break
                _, chunk_size, chunk_data = chunk
                self.f_out.write(chunk_data)
                if self.callback is not None:
                    self.callback(chunk_data)

                received_size += chunk_size

            # check transfer status
            if received_size >= file_size:
                break

            # check chunks queues
            if not self.requested_chunks and len(self.received_chunks) >= self._DOWNLOAD_MAX_REQUESTS:
                raise ValueError("SFTP communication error. The queue with requested file chunks is empty and"
                                 "the received chunks queue is full and cannot be consumed.")

        return received_size

    def _sftp_async_read_request(self, fileobj, file_handle, offset, size):
        sftp_client = self.f_in.sftp

        with sftp_client._lock:
            num = sftp_client.request_number

            msg = Message()
            msg.add_int(num)
            msg.add_string(file_handle)
            msg.add_int64(offset)
            msg.add_int(size)

            sftp_client._expecting[num] = fileobj
            sftp_client.request_number += 1

        sftp_client._send_packet(CMD_READ, msg)
        return num

    def _async_response(self, t, msg, num):
        if t == CMD_STATUS:
            # save exception and re-raise it on next file operation
            try:
                self.f_in.sftp._convert_status(msg)
            except Exception as e:
                self.saved_exception = e
            return
        if t != CMD_DATA:
            raise SFTPError("Expected data")
        data = msg.get_string()

        chunk_data = self.requested_chunks.pop(num, None)
        if chunk_data is None:
            return

        # save chunk
        offset, size = chunk_data

        if size != len(data):
            raise SFTPError(f"Invalid data block size. Expected {size} bytes, but it has {len(data)} size")
        self.received_chunks[offset] = (offset, size, data)

    def _check_exception(self):
        """if there's a saved exception, raise & clear it"""
        if self.saved_exception is not None:
            x = self.saved_exception
            self.saved_exception = None
            raise x


def download_file(sftp_client: SFTPClient, remote_path: str, local_path: str, callback=None):
    """
    Helper function to download remote file via sftp.
    It contains a fix for a bug that prevents a large file downloading with :meth:`paramiko.SFTPClient.get`
    Note: this function relies on some private paramiko API and has been tested with paramiko 2.7.1.
          So it may not work with other paramiko versions.
    :param sftp_client: paramiko sftp client
    :param remote_path: remote file path
    :param local_path: local file path
    :param callback: optional data callback
    """
    remote_file_size = sftp_client.stat(remote_path).st_size

    with sftp_client.open(remote_path, 'rb') as f_in, open(local_path, 'wb') as f_out:
        _SFTPFileDownloader(
            f_in=f_in,
            f_out=f_out,
            callback=callback
        ).download()

    local_file_size = os.path.getsize(local_path)
    if remote_file_size != local_file_size:
        raise IOError(f"file size mismatch: {remote_file_size} != {local_file_size}")


def main():

    start_time = time.time()
    # public sftp server
    host = 'DOTS.datalabusa.com'
    port = 22
    username = 'ex_Avant'
    password = 'Et#h@ut8Ec2F'
    remote_file_path = 'To_Datalab/AV_SUPP_FUNDED_07212024.csv.gpg'
#     local_file_path = join(dirname(__file__), 'DL_AVT_INTERIMFILE2_202408_HI.csv.pgp')
    local_file_path = '/tmp/AV_SUPP_FUNDED_07212024.csv.gpg'

    transport = Transport((host, port))
    transport.set_keepalive(30)
    transport.connect(
        username=username,
        password=password,
    )
    
    with SFTPClient.from_transport(transport) as sftp_client:
        progress_size = 0
        total_size = 0
        step_size = 4 * 1024 * 1024

        def progress_callback(data):
            nonlocal progress_size, total_size
            progress_size += len(data)
            total_size += len(data)
            while progress_size >= step_size:
                logger.info(f"{total_size // (1024 ** 2)} MB has been downloaded")
                progress_size -= step_size
        
        download_file(
            sftp_client=sftp_client,
            remote_path=remote_file_path,
            local_path=local_file_path,
            callback=progress_callback
        )
    download_time = time.time() - start_time
    print(f"File downloaded successfully to {local_file_path} in {download_time:.2f} seconds")

    # s4 upload
    s3_bucket_name = 'avant-partner01-landing'
    s3_object_key = 'datalabs/drop/interim/AV_SUPP_FUNDED_07212024.csv.gpg'


    # Initialize the S3 client
    s3_client = boto3.client('s3')

    # Step 1: Initiate the multipart upload
    response = s3_client.create_multipart_upload(Bucket=s3_bucket_name, Key=s3_object_key)
    upload_id = response['UploadId']
    print(f'Upload ID: {upload_id}')

    # Step 2: Upload parts
    part_size = 10 * 1024 * 1024  # 5 MB
    part_info = {'Parts': []}
    part_number = 1

    with open(local_file_path, 'rb') as file:
        while True:
            data = file.read(part_size)
            if not data:
                break

            # Generate pre-signed URL for the part
            presigned_url = s3_client.generate_presigned_url(
                ClientMethod='upload_part',
                Params={
                    'Bucket': s3_bucket_name,
                    'Key': s3_object_key,
                    'UploadId': upload_id,
                    'PartNumber': part_number,
                }
            )

            # Upload the part
            response = requests.put(presigned_url, data=data)
            print(response)
            etag = response.headers['ETag']

            # Keep track of uploaded part details
            part_info['Parts'].append({
                'PartNumber': part_number
            })

            print(f'Uploaded part {part_number} ')
            part_number += 1

    # Step 3: Complete the multipart upload
    response = s3_client.complete_multipart_upload(
        Bucket=s3_bucket_name,
        Key=s3_object_key,
        UploadId=upload_id,
        MultipartUpload=part_info
    )
    print(f'Multipart upload completed: {response}')

    # Clean up local file
    os.remove(local_file_path)
    
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")
    main()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
main()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'etag'
Traceback (most recent call last):
  File "<stdin>", line 218, in main
  File "/usr/local/lib/python3.7/site-packages/requests/structures.py", line 52, in __getitem__
    return self._store[key.lower()][1]
KeyError: 'etag'



In [5]:
# s4 upload
s3_bucket_name = 'avant-partner01-landing'
s3_object_key = 'datalabs/drop/interim/AV_SUPP_FUNDED_07212024.csv.gpg'


# Initialize the S3 client
s3_client = boto3.client('s3')

# Step 1: Initiate the multipart upload
response = s3_client.create_multipart_upload(Bucket=s3_bucket_name, Key=s3_object_key)
upload_id = response['UploadId']
print(f'Upload ID: {upload_id}')

# Step 2: Upload parts
part_size = 10 * 1024 * 1024  # 5 MB
part_info = {'Parts': []}
part_number = 1


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Upload ID: wPdE1WJlAVBbbbQvxgTwQ2HDJezOf9u4AZOlRL9o29cCzl9ap5Xeih1bY3Gr6WawAMxeRLSk_ANEaEn573H71g.1nvz7UL61BxjCw4KAkN729bJGo8gzpGGyS7votcz0STfmowpi7cBB_MFcFBbdFKfxi.m2hdk5bTD1HHrVHhc-

In [6]:
with open('/tmp/AV_SUPP_FUNDED_07212024.csv.gpg', 'rb') as file:
    while True:
        data = file.read(part_size)
        if not data:
            break

        # Generate pre-signed URL for the part
        presigned_url = s3_client.generate_presigned_url(
            ClientMethod='upload_part',
            Params={
                'Bucket': s3_bucket_name,
                'Key': s3_object_key,
                'UploadId': upload_id,
                'PartNumber': part_number,
            }
        )


        # Upload the part
        response = requests.put(presigned_url, data=data)
        print(response)
#         etag = response.headers['ETag']

        # Keep track of uploaded part details
        part_info['Parts'].append({
            'PartNumber': part_number
        })

        print(f'Uploaded part {part_number} ')
        part_number += 1

# Step 3: Complete the multipart upload
response = s3_client.complete_multipart_upload(
    Bucket=s3_bucket_name,
    Key=s3_object_key,
    UploadId=upload_id,
    MultipartUpload=part_info
)
print(f'Multipart upload completed: {response}')

# Clean up local file
os.remove(local_file_path)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred (MalformedXML) when calling the CompleteMultipartUpload operation: The XML you provided was not well-formed or did not validate against our published schema
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/botocore/client.py", line 508, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/usr/local/lib/python3.7/site-packages/botocore/client.py", line 915, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (MalformedXML) when calling the CompleteMultipartUpload operation: The XML you provided was not well-formed or did not validate against our published schema



In [18]:
    with open(local_file_path, 'rb') as file:
        while True:
            data = file.read(part_size)
            if not data:
                break

            # Generate pre-signed URL for the part
            presigned_url = s3_client.generate_presigned_url(
                ClientMethod='upload_part',
                Params={
                    'Bucket': s3_bucket_name,
                    'Key': s3_object_key,
                    'UploadId': upload_id,
                    'PartNumber': part_number,
                }
            )

            # Upload the part
            response = requests.put(presigned_url, data=data)
            print(response)
            etag = response.headers['ETag']

            # Keep track of uploaded part details
            part_info['Parts'].append({
                'PartNumber': part_number,
                'ETag': etag
            })

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'local_file_path' is not defined
Traceback (most recent call last):
NameError: name 'local_file_path' is not defined

